# Stat198: Group Instance Cost Function #

## Setup

In [1]:
# imports
import json
import pandas as pd
from pprint import pprint

## Amazon .json file format
    reviewerID - ID of the reviewer, e.g. A2SUAM1J3GNN3B
    asin - ID of the product, e.g. 0000013714
    reviewerName - name of the reviewer
    helpful - helpfulness rating of the review, e.g. 2/3
    reviewText - text of the review
    overall - rating of the product
    summary - summary of the review
    unixReviewTime - time of the review (unix time)
    reviewTime - time of the review (raw)

In [2]:
# parse data downloaded from http://jmcauley.ucsd.edu/data/amazon/

file_path = 'C:/Users/Gavin/Desktop/Cell_Phones_and_Accessories_5.json'

df = pd.read_json(file_path, 'r', lines=True)

TypeError: read_json() got an unexpected keyword argument 'lines'

In [12]:
# above cells did not work for me so I used that to parse the data and added the gz file to the git repo

# parse data downloaded from http://jmcauley.ucsd.edu/data/amazon/

import pandas as pd 
import gzip 

def parse(path): 
    g = gzip.open(path, 'rb') 
    for l in g: 
        yield eval(l) 
        
def getDF(path): 
    i = 0 
    df = {}
    for d in parse(path): 
        df[i] = d 
        i += 1 
    return pd.DataFrame.from_dict(df, orient='index') 

df = getDF('reviews_Cell_Phones_and_Accessories_5.json.gz')

NameError: name 'head' is not defined

In [13]:
df.head()

,summary,asin,unixReviewTime,helpful,reviewerID,reviewText,reviewerName,overall,reviewTime
0,Looks Good,120401325X,1400630400,"[0, 0]",A30TL5EWN6DFXT,They look good and stick good! I just don't li...,christina,4.0,"05 21, 2014"
1,Really great product.,120401325X,1389657600,"[0, 0]",ASY55RVNIL0UD,These stickers work like the review says they ...,emily l.,5.0,"01 14, 2014"
2,LOVE LOVE LOVE,120401325X,1403740800,"[0, 0]",A2TMXE2AFO7ONB,These are awesome and make my phone look so st...,Erica,5.0,"06 26, 2014"
3,Cute!,120401325X,1382313600,"[4, 4]",AWJ0WZQYMYFQ4,Item arrived in great time and was in perfect ...,JM,4.0,"10 21, 2013"
4,leopard home button sticker for iphone 4s,120401325X,1359849600,"[2, 3]",ATX7CZYFXI1KW,"awesome! stays on, and looks great. can be use...",patrice m rogoza,5.0,"02 3, 2013"


In [14]:
#Preprocessing data

def reviews_to_word_split(reviews):
    """
    Given a set of reviews, converts each to lowercase, splits them into sentences and splits each sentence into words. Strips 
    all commas, periods, and dashes from each review.
    """
    reviews = reviews.str.lower()
    reviews = reviews.str.replace("[,|-]", "")
    reviews = reviews.str.split("[.*\.|\!|?]")
    return [[sentance.split() for sentance in review] for review in reviews]

df.loc[:, "reviewText"] = reviews_to_word_split(df.loc[:, "reviewText"])

In [15]:
df.head()

,summary,asin,unixReviewTime,helpful,reviewerID,reviewText,reviewerName,overall,reviewTime
0,Looks Good,120401325X,1400630400,"[0, 0]",A30TL5EWN6DFXT,"[[they, look, good, and, stick, good], [i, jus...",christina,4.0,"05 21, 2014"
1,Really great product.,120401325X,1389657600,"[0, 0]",ASY55RVNIL0UD,"[[these, stickers, work, like, the, review, sa...",emily l.,5.0,"01 14, 2014"
2,LOVE LOVE LOVE,120401325X,1403740800,"[0, 0]",A2TMXE2AFO7ONB,"[[these, are, awesome, and, make, my, phone, l...",Erica,5.0,"06 26, 2014"
3,Cute!,120401325X,1382313600,"[4, 4]",AWJ0WZQYMYFQ4,"[[item, arrived, in, great, time, and, was, in...",JM,4.0,"10 21, 2013"
4,leopard home button sticker for iphone 4s,120401325X,1359849600,"[2, 3]",ATX7CZYFXI1KW,"[[awesome], [stays, on, and, looks, great], [c...",patrice m rogoza,5.0,"02 3, 2013"
